In [ ]:
import warnings
# 경고 메시지를 무시하고 숨기거나
warnings.filterwarnings(action='ignore')

#결과 여러개 뜨게 하기
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
from cvxopt import matrix
from cvxopt.solvers import qp

import arch

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss


## Data Preprocessing for security_selection
Extract only the top 10 cumulative returns over 10 years

In [ ]:
#dataframe으로 변환
df_stock_data = pd.read_excel('./dataset.xlsx',index_col=0)

In [ ]:
#전략 
df_train = df_stock_data.iloc[1:122,:]
#월별 수익률(2000~2009)
df_profit = df_train.pct_change()

#상위 누적수익률로 포트폴리오 짜는 방법
df_profit_acc = (1+df_profit).cumprod()-1
select_stocks = df_profit_acc.iloc[-1,:].sort_values(ascending = False)
# select_stocks = list(select_stocks[:25].index)

In [ ]:
#상위 누적수익률 10개 중위 10개 하위 5개 랜덤 섞기
select_stocks

In [ ]:
# # 랜덤으로 짜는 방법
# import random
# columns = list(df_profit.columns)
# select_stocks = random.sample(columns, 25)

### 1. 예상 수익률과 종목 변동성을 각각 arima와 garch로 구함
### 2. makowitz 공식에 대입해서 weight값 구함
### 3. weight값을 토대로 월별 수익률을 기록 이후 sliding window 방식으로 백테스팅

In [ ]:
def predict_with_arima(stock_rate):
    model = ARIMA(stock_rate.diff(), order=(1,0,1)).fit()
    forcast = model.forecast(steps=1)
    return np.array(forcast)[0] + stock_rate[-1]

In [ ]:
def predict_with_garch(stock_rate):
    model = arch.arch_model(stock_rate,vol='GARCH', p=1,q=1).fit()
    forcast = model.forecast(horizon=1)
    return np.sqrt(forcast.variance[-1:].values[0][0])
    

In [ ]:
# #저장한걸 불러오기
# stock_list = pd.read_excel('num_is_25_r_min_001_random2.xlsx')
# 
# select_stocks = list(stock_list.columns)
# select_stocks

In [ ]:
#포트폴리오 연수익을 구하는 함수
def calculate_return_rate(select_stocks, r_min):    
    df_month_profit = df_stock_data.iloc[1:,:][select_stocks].pct_change().dropna()
    num = len(select_stocks)
    return_rate = []

    index = list(df_month_profit.index)[120:len(df_month_profit)]
    #walk forward analysis
    #총 데이터 갯수 252개(0~251) 132번 테스트 
    for i in range(0,len(df_month_profit)-120):
        security_selection_train_return = df_month_profit.iloc[i:120+i,:]
        security_selection_test_return = df_month_profit.iloc[120+i,:]
    
        corr = security_selection_train_return.corr().to_numpy()
        mean_list = []
        sigma_list = []
        for j in range(num):
            target = security_selection_train_return.iloc[:,j]
            forcast_return_result = predict_with_arima(target)
            mean_list.append(forcast_return_result)
    
            forcast_vot_result = predict_with_garch(target)
            sigma_list.append(forcast_vot_result)
    
        weight = calculate_weight(mean_list,sigma_list,corr,num,r_min)
        weight = np.transpose(weight)
        x = matrix(security_selection_test_return)
        month_return = weight @ x
        return_rate.append(month_return[0][0])
    
    return_rate = pd.DataFrame(return_rate)
    return_rate.index = index
    
    return df_month_profit,return_rate

In [ ]:
def calculate_weight(mean_list,sigma_list,corr,num,r_min):
    #공분산 행렬 만들기 sigma @ T(sigma) * corr   
    Mean = matrix([mean_list])
    sigma = matrix([sigma_list])
    sigma = sigma@np.transpose(sigma)
    cov = matrix(sigma*corr)

    G = matrix(np.concatenate((-np.transpose(Mean), -np.identity(num)), 0))
    h = matrix(np.concatenate((-np.ones((1,1))*r_min, np.zeros((num,1))), 0))
    A = matrix(1.0, (1,num))
    b = matrix(1.0)
    q = matrix(np.zeros((num,1)))
    sol = qp(cov, q, G, h, A ,b)
    return sol['x']

In [ ]:
return_rate = calculate_return_rate(select_stocks,0.01)

In [ ]:
df_month_profit, return_rate = return_rate

In [ ]:
#종목 내보내기
# df_month_profit.to_excel('best.xlsx')

In [ ]:
# 누적수익률
df_profit_accumulate = (1+return_rate).cumprod()-1

In [ ]:
#코스피 표시
df_kospi_data = pd.read_excel('./kospi.xlsx',index_col=0)
df_kospi_data_pct_return = df_kospi_data.pct_change().dropna()
df_kospi_data_accumulate_return = (1+df_kospi_data_pct_return).cumprod()-1
df_kospi_data_accumulate_return.index = df_profit_accumulate.index

In [ ]:
def calculate_value(return_rate):
    acc_return_rate = (1+return_rate)
    initial_price = 100
    acc_return_rate.iloc[0,:] = acc_return_rate.iloc[0,:] * initial_price
    acc_return_rate = acc_return_rate.cumprod()
    
    return acc_return_rate
    

In [ ]:
def cal_sharpe(return_rate):
    acc_return_rate = calculate_value(return_rate)
    initial_price = 100
    last_price = acc_return_rate.iloc[-1,:][0]
    
    annualized_return = np.power(last_price/initial_price,1/11)-1
    annualized_std = return_rate.std()*np.sqrt(12)
    #샤프지수
    sharp = annualized_return / annualized_std
    print(f'annualized_return : {annualized_return}\nannualized_std : {annualized_std[0]}\nsharp: {sharp[0]}')
    
    

In [ ]:
cal_sharpe(return_rate)

In [ ]:
cal_sharpe(df_kospi_data_pct_return)

In [ ]:
df_kospi_data_pct_return.index = return_rate.index

plt.plot(calculate_value(df_kospi_data_pct_return), label='kospi')
plt.plot(calculate_value(return_rate), label='portfolio')
plt.xlabel('year')
plt.ylabel('acc_value')
plt.legend()
